In [1]:
!pip install transformers

In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd

## Get data from this link:- (https://drive.google.com/file/d/11O1urBg0Lg7nJa3uM6pXlf6IjK28E033/view?usp=sharing)

# New Section

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()


,BOS Sir I created a new env with python 3.7 but the library for which I got error &quot;mkl-random==1.1.0&quot; is not compatible with python 3.7 EOS,O O O O O O O O O O O O O O O O O O O O O O O O O O,need_help
0,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help
1,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings
2,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great
3,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link
4,BOS Understood EOS,O O O,great


In [ ]:
df.columns=['A','B','C']

In [ ]:
df.head()

,A,B,C
0,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help
1,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings
2,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great
3,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link
4,BOS Understood EOS,O O O,great


In [ ]:
df['C'].value_counts()

need_help                           780
O                                   609
Greetings                           331
Thank_you                           284
good_bye                            169
                                   ... 
list_comprehensions                   1
usecase_document_contains_letter      1
aws_deployment_error                  1
python_super_class                    1
azure_luis_deployment                 1
Name: C, Length: 151, dtype: int64

In [ ]:
possible_labels = df.C.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'CNN_code_error': 127,
 'Dashboard_access': 39,
 'Dashboard_details': 83,
 'EDA_code': 111,
 'Exception_builtin_errors': 97,
 'GCP_chatbot': 78,
 'Greetings': 1,
 'NewBatch_details': 54,
 'O': 8,
 'PlotlyRequestError': 33,
 'Plotly_installation': 60,
 'Thank_you': 13,
 'anaconda_jnotebook_run': 89,
 'anaconda_macos': 117,
 'anaconda_navigator_open': 21,
 'anaconda_prompt': 126,
 'assignment_by_github': 150,
 'assignment_data_visulization': 57,
 'assignment_doubt': 7,
 'assignment_evaluation': 96,
 'assignment_exception_handling': 128,
 'assignment_issue': 131,
 'assignment_link': 86,
 'assignment_list_comprehension': 134,
 'assignment_matplotlib': 113,
 'assignment_numpy': 88,
 'assignment_pandas': 6,
 'assignment_pandas_link': 77,
 'assignment_powerbi': 123,
 'assignment_python': 43,
 'assignment_stats': 53,
 'assignment_submittion': 50,
 'assingment_uploading_error': 109,
 'aws_deployment_error': 112,
 'aws_lex_account_query': 147,
 'aws_lex_smtp_error': 116,
 'azure_luis_500_error'

In [ ]:
df['label'] = df.C.replace(label_dict)

In [ ]:
df.head()

,A,B,C,label
0,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help,0
1,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings,1
2,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great,2
3,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link,3
4,BOS Understood EOS,O O O,great,2


In [ ]:
df.tail()

,A,B,C,label
3508,BOS it is not one to one resume but some advic...,O O O O O O O O O O O O O O O O,need_help,0
3509,BOS hi good noon EOS,O O O O O,Greetings,1
3510,BOS now what will year contain EOS,O O O O O O O,code_error,5
3511,BOS I have extracted it but didn&apos;t found ...,O O O O O O O O O O,need_help,0
3512,BOS file name or extension path name EOS,O O O O O O O O,need_help,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['C', 'label', 'data_type']).count()

A   B
C                          label data_type        
CNN_code_error             127   train       1   1
Dashboard_access           39    train      15  15
                                 val         2   2
Dashboard_details          83    train       3   3
                                 val         1   1
...                                         ..  ..
yes_reply                  9     train      19  19
                                 val         3   3
youtube_video_availability 35    train       5   5
zoom_mute_disable_reply    119   train       1   1
zoom_screen_share          148   train       1   1

[216 rows x 2 columns]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].A.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].A.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)


(2986, 527)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)


cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/Models_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')



Epoch 1
Training loss: 1.6121188362856511
Validation loss: 1.9203494982793927
F1 Score (Weighted): 0.5292682399883665



Epoch 2
Training loss: 1.6150034834625178
Validation loss: 1.9203494982793927
F1 Score (Weighted): 0.5292682399883665



Epoch 3
Training loss: 1.6146310993159811
Validation loss: 1.9203494982793927
F1 Score (Weighted): 0.5292682399883665


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.load_state_dict(torch.load('/content/Models_1.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)


In [ ]:
accuracy_per_class(predictions, true_vals)

Class: need_help
Accuracy: 98/119

Class: Greetings
Accuracy: 48/51

Class: great
Accuracy: 0/11

Class: community_class_link
Accuracy: 0/1

Class: welocme_reply
Accuracy: 1/9

Class: code_error
Accuracy: 13/30

Class: assignment_doubt
Accuracy: 0/2

Class: O
Accuracy: 88/101

Class: yes_reply
Accuracy: 0/3

Class: support_ticket_generate
Accuracy: 12/28

Class: more_query
Accuracy: 0/3

Class: need_skype_call
Accuracy: 0/5

Class: Thank_you
Accuracy: 26/34

Class: stats
Accuracy: 0/2

Class: enter_query
Accuracy: 0/3

Class: internship_query
Accuracy: 5/5

Class: sure
Accuracy: 0/8

Class: class_link
Accuracy: 0/1

Class: power-bi_tableau_dashboard
Accuracy: 0/2

Class: deployment
Accuracy: 0/1

Class: image_scrapper
Accuracy: 0/2

Class: powerbi_query
Accuracy: 0/1

Class: resume_query
Accuracy: 0/2

Class: good_bye
Accuracy: 20/28

Class: course_details
Accuracy: 0/3

Class: okay_reply
Accuracy: 0/2

Class: batch_details
Accuracy: 0/3

Class: pyspark
Accuracy: 0/2

Class: seaborn
Ac